In [6]:
import os
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
import joblib
from sklearn.model_selection import StratifiedShuffleSplit
from imblearn.over_sampling import SMOTE
from sklearn.metrics import accuracy_score


# Ensure models folder exists
os.makedirs("models", exist_ok=True)

# Load data
df = pd.read_csv("data/gender_affirming_surgery_risk_dataset (1).csv")

# Handle missing values
imputer = SimpleImputer(strategy='most_frequent')
df_imputed = pd.DataFrame(imputer.fit_transform(df), columns=df.columns)

# Encode categorical features
label_encoders = {}
for col in df_imputed.select_dtypes(include=['object']).columns:
    le = LabelEncoder()
    df_imputed[col] = le.fit_transform(df_imputed[col])
    label_encoders[col] = le

# Scale features
scaler = StandardScaler()
X = df_imputed.drop(columns=["PostOp_Risk_Level", "Complications"])
y_risk = df_imputed["PostOp_Risk_Level"]
y_complications = df_imputed["Complications"]

X_scaled = scaler.fit_transform(X)

# Train-test split
X_train, X_test, y_risk_train, y_risk_test = train_test_split(X_scaled, y_risk, test_size=0.2, random_state=42)
_, _, y_comp_train, y_comp_test = train_test_split(X_scaled, y_complications, test_size=0.2, random_state=42)

# Train Risk model


sss_risk = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
for train_idx, test_idx in sss_risk.split(X_scaled, y_risk):
    X_risk_train, X_risk_test = X_scaled[train_idx], X_scaled[test_idx]
    y_risk_train, y_risk_test = y_risk.iloc[train_idx], y_risk.iloc[test_idx]

# Apply SMOTE to risk training data
smote = SMOTE(random_state=42)
X_risk_train_sm, y_risk_train_sm = smote.fit_resample(X_risk_train, y_risk_train)

# Train risk model with class_weight='balanced'
risk_model = RandomForestClassifier(class_weight='balanced', random_state=42)
risk_model.fit(X_risk_train_sm, y_risk_train_sm)
y_risk_pred = risk_model.predict(X_risk_test)
print("Risk Prediction Report:")
print(classification_report(y_risk_test, y_risk_pred))
print("Accuracy:", accuracy_score(y_risk_test, y_risk_pred))

# Train Complication model
comp_model = RandomForestClassifier(random_state=42)
comp_model.fit(X_train, y_comp_train)
comp_preds = comp_model.predict(X_test)
print("Complications Prediction Report:\n", classification_report(y_comp_test, comp_preds))
print("Accuracy:", accuracy_score(y_comp_test, comp_preds))

# Save models and preprocessors


Risk Prediction Report:
              precision    recall  f1-score   support

           0       1.00      0.70      0.82        10
           1       0.73      0.84      0.78        19
           2       0.81      0.81      0.81        31

    accuracy                           0.80        60
   macro avg       0.84      0.78      0.80        60
weighted avg       0.81      0.80      0.80        60

Accuracy: 0.8
Complications Prediction Report:
               precision    recall  f1-score   support

           0       0.00      0.00      0.00         8
           1       0.47      1.00      0.64        28
           2       0.00      0.00      0.00         9
           3       0.00      0.00      0.00         2
           4       0.00      0.00      0.00         9
           5       0.00      0.00      0.00         4

    accuracy                           0.47        60
   macro avg       0.08      0.17      0.11        60
weighted avg       0.22      0.47      0.30        60

Accu

C:\Users\my pc\AppData\Roaming\Python\Python311\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\my pc\AppData\Roaming\Python\Python311\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\my pc\AppData\Roaming\Python\Python311\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} i

In [7]:
joblib.dump(risk_model, "api/models/risk_model.pkl")
joblib.dump(comp_model, "api/models/comp_model.pkl")
joblib.dump(scaler, "api/models/scaler.pkl")
joblib.dump(label_encoders, "api/models/label_encoders.pkl")

['api/models/label_encoders.pkl']